In [73]:
import numpy as np
import torch
import abc
import glob
import os
import shutil
import cv2
import pandas as pd
from PIL import Image
from skimage.io import imread
from natsort import natsorted

from sray.utils.base_utils import downsample_gaussian_blur, pose_inverse
from sray.dataset.semantic_utils import PointSegClassMapping

In [2]:
data = imread('/home/chengshun.wang/pjs/Semantic-Ray/data/scannet/scene0000_00/color/0.jpg')
data.shape

(968, 1296, 3)

In [98]:
scannet_train_scans_320 = np.loadtxt('configs/scannetv2_train_split.txt',dtype=str).tolist()
scannet_val_scans_320 = np.loadtxt('configs/scannetv2_val_split.txt',dtype=str).tolist()
scannet_test_scans_320 = np.loadtxt('configs/scannetv2_test_split.txt',dtype=str).tolist()
scannet_train_scans_320 = [i.split('/')[1] for i in scannet_train_scans_320]
scannet_test_scans_320 = [i.split('/')[1] for i in scannet_test_scans_320]
scannet_val_scans_320 = [i.split('/')[1] for i in scannet_val_scans_320]
split2file_list={
    'train':scannet_train_scans_320,
    'test':scannet_test_scans_320,
    'val':scannet_val_scans_320,
}

In [114]:
image_size = 320
root_dir = '/mnt/data/chengshun.wang/scannet_2d'
data_dir = 'data/scannet/'
ratio = image_size / 1296
h, w = int(ratio*972), int(image_size)
h, w = 240, int(image_size)


In [18]:
mapping_file = 'data/scannet/scannetv2-labels.combined.tsv'
mapping_file = pd.read_csv(mapping_file, sep='\t', header=0)
scan_ids = mapping_file['id'].values
nyu40_ids = mapping_file['nyu40id'].values
scan2nyu = np.zeros(max(scan_ids) + 1, dtype=np.int32)
for i in range(len(scan_ids)):
    scan2nyu[scan_ids[i]] = nyu40_ids[i]
scan2nyu = scan2nyu
label_mapping = PointSegClassMapping(
    valid_cat_ids=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                   11, 12, 14, 16, 24, 28, 33, 34, 36, 39],
    max_cat_id=40
)

In [120]:
def get_image(path):
    img = imread(path)
    if w != 1296:
        img = cv2.resize(downsample_gaussian_blur(
            img, ratio), (w, h), interpolation=cv2.INTER_LINEAR)
    return img

In [20]:
os.makedirs('/mnt/data/chengshun.wang/scannet_2d',exist_ok=True)
os.makedirs('/mnt/data/chengshun.wang/scannet_2d/img_dir/train',exist_ok=True)
os.makedirs('/mnt/data/chengshun.wang/scannet_2d/img_dir/val',exist_ok=True)
os.makedirs('/mnt/data/chengshun.wang/scannet_2d/img_dir/test',exist_ok=True)
os.makedirs('/mnt/data/chengshun.wang/scannet_2d/ann_dir/train',exist_ok=True)
os.makedirs('/mnt/data/chengshun.wang/scannet_2d/ann_dir/val',exist_ok=True)
os.makedirs('/mnt/data/chengshun.wang/scannet_2d/ann_dir/test',exist_ok=True)


In [127]:
def prepare_dataset_by_id(scene_id,split):
    
    rgb_paths = sorted([x for x in glob.glob(os.path.join(
                data_dir,scene_id, "color", "*")) if (x.endswith(".jpg") or x.endswith(".png"))])
    ano_paths = sorted([x for x in glob.glob(os.path.join(
                data_dir,scene_id, "label-filt", "*")) if (x.endswith(".jpg") or x.endswith(".png"))])
    assert len(rgb_paths) == len(ano_paths)
    for ind,rgb_path in enumerate(rgb_paths):
        img = get_image(rgb_path)
        
        label = Image.open(ano_paths[ind])
        label = np.asarray(label, dtype=np.int32)
        label = np.ascontiguousarray(label)
        label = cv2.resize(label, (w, h), interpolation=cv2.INTER_NEAREST)
        label = label.astype(np.int32)
        label = scan2nyu[label]
        label = label_mapping(label)
        basename = str(scene_id) +'_'+ os.path.basename(ano_paths[ind]).split('.')[0]
        # print(basename)
        Image.fromarray(label.astype('uint8'), mode='L').save(os.path.join(root_dir,'ann_dir',split,basename+'.png'))
        rgb_file_name = str(scene_id) +'_'+ os.path.basename(rgb_path)
        Image.fromarray(img.astype('uint8')).save(os.path.join(root_dir,'img_dir',split,rgb_file_name))
        
        # load_ = np.asarray(Image.open(os.path.join(root_dir,'ann_dir',split,basename+'.png')), dtype=np.int32)
        # assert (load_==label).all()
        rgb_file_name = str(scene_id) +'_'+ os.path.basename(rgb_path)
        # shutil.copy(rgb_path,os.path.join(root_dir,'img_dir',split,rgb_file_name))
        
        

In [128]:
prepare_dataset_by_id('scene0160_00','test')

In [96]:
from multiprocessing import Pool

In [129]:
for split in ['train','val','test']:
    with Pool(processes=32) as pool:
        # for scene_id in split2file_list[split]:
        #     input_data = [scene_id,split]
        files = split2file_list[split]
        input_data = list(zip(files,[split]*len(files)))
        print(input_data)
        results = pool.starmap(prepare_dataset_by_id, input_data)

[('scene0000_00', 'train'), ('scene0002_00', 'train'), ('scene0003_00', 'train'), ('scene0004_00', 'train'), ('scene0005_00', 'train'), ('scene0006_00', 'train'), ('scene0007_00', 'train'), ('scene0008_00', 'train'), ('scene0009_00', 'train'), ('scene0010_00', 'train'), ('scene0011_00', 'train'), ('scene0012_00', 'train'), ('scene0013_00', 'train'), ('scene0014_00', 'train'), ('scene0015_00', 'train'), ('scene0016_00', 'train'), ('scene0017_00', 'train'), ('scene0018_00', 'train'), ('scene0019_00', 'train'), ('scene0020_00', 'train'), ('scene0021_00', 'train'), ('scene0022_00', 'train'), ('scene0023_00', 'train'), ('scene0024_00', 'train'), ('scene0025_00', 'train'), ('scene0026_00', 'train'), ('scene0027_00', 'train'), ('scene0028_00', 'train'), ('scene0029_00', 'train'), ('scene0030_00', 'train'), ('scene0031_00', 'train'), ('scene0032_00', 'train'), ('scene0033_00', 'train'), ('scene0034_00', 'train'), ('scene0035_00', 'train'), ('scene0036_00', 'train'), ('scene0037_00', 'train'), 

In [133]:
path = '/mnt/data/chengshun.wang/scannet_2d/img_dir/train'
for i in os.listdir(path):
    img = np.asarray(Image.open(os.path.join(path,i)))
    assert img.shape == (240,320,3),f"img.shape"

In [111]:
len(os.listdir('/mnt/data/chengshun.wang/scannet_2d/ann_dir/train'))

313591

In [112]:
len(os.listdir('/mnt/data/chengshun.wang/scannet_2d/ann_dir/val'))

5089

In [113]:
len(os.listdir('/mnt/data/chengshun.wang/scannet_2d/ann_dir/test'))

13224

In [95]:
ls /mnt/data/chengshun.wang/scannet_2d/img_dir/test

scene0160_00_0.jpg  scene0160_00_637.jpg


In [94]:
!ls /mnt/data/chengshun.wang/scannet_2d/ann_dir/

test  train  val


In [60]:
data = np.random.randint(0,100,(10,10),dtype=np.uint8)

In [61]:
Image.fromarray(data, mode='L').save('test.png')

In [62]:
data_load = Image.open('test.png')

In [66]:
data_load = np.asarray(data_load,dtype=np.int32)

In [67]:
data_load

array([[38, 36, 49, 37, 43, 98, 35, 35, 95, 18],
       [49, 73, 13, 22, 12, 90, 44, 26, 26, 33],
       [ 9, 18, 72, 66, 12, 87, 19, 64, 19, 10],
       [72, 96, 22, 91, 85, 50, 51, 37, 16, 62],
       [71, 42, 68, 13, 50, 88, 55, 30, 21, 89],
       [ 3, 42, 88, 78, 75, 37, 85, 50, 35, 36],
       [54,  4, 44,  2, 81, 12, 17, 88, 63, 74],
       [63, 49, 72, 87, 68,  8, 26, 61, 21, 24],
       [38, 38, 72,  1, 48, 15, 42, 75, 49, 69],
       [61, 14, 54, 94, 41, 34, 52, 97, 40, 36]], dtype=int32)

In [68]:
data

array([[38, 36, 49, 37, 43, 98, 35, 35, 95, 18],
       [49, 73, 13, 22, 12, 90, 44, 26, 26, 33],
       [ 9, 18, 72, 66, 12, 87, 19, 64, 19, 10],
       [72, 96, 22, 91, 85, 50, 51, 37, 16, 62],
       [71, 42, 68, 13, 50, 88, 55, 30, 21, 89],
       [ 3, 42, 88, 78, 75, 37, 85, 50, 35, 36],
       [54,  4, 44,  2, 81, 12, 17, 88, 63, 74],
       [63, 49, 72, 87, 68,  8, 26, 61, 21, 24],
       [38, 38, 72,  1, 48, 15, 42, 75, 49, 69],
       [61, 14, 54, 94, 41, 34, 52, 97, 40, 36]], dtype=uint8)

In [69]:
np.asarray(Image.open('data/scannet/scene0000_00/label-filt/100.png'), dtype=np.int32).max()

103

In [41]:
from PIL import Image
import numpy as np

# 创建形状为(100, 100, 1)的示例语义分割图
semantic_segmentation_tensor = np.random.randint(0, 256, size=(100, 100, 1), dtype=np.uint8)

# 将numpy数组转换为PIL图像
semantic_segmentation_image = Image.fromarray(semantic_segmentation_tensor.squeeze(), mode='L')

# 保存PIL图像为PNG文件
semantic_segmentation_image.save('semantic_segmentation.png')

In [42]:
loaded_image = Image.open('semantic_segmentation.png')

# 将加载的图像转换为NumPy数组
loaded_semantic_segmentation_tensor = np.array(loaded_image)

# 如果需要将通道数恢复为 1（从灰度图像中加载）
if len(loaded_semantic_segmentation_tensor.shape) == 2:
    loaded_semantic_segmentation_tensor = loaded_semantic_segmentation_tensor[:, :, np.newaxis]

In [44]:
semantic_segmentation_tensor.flatten()

array([  1, 227,  13, ..., 182, 205,  33], dtype=uint8)

In [45]:
loaded_semantic_segmentation_tensor.flatten()

array([  1, 227,  13, ..., 182, 205,  33], dtype=uint8)

In [71]:
str(111)+os.path.basename('xxx/yyy/zzz.png')

'111zzz.png'

In [72]:
data.astype('uint8')

array([[38, 36, 49, 37, 43, 98, 35, 35, 95, 18],
       [49, 73, 13, 22, 12, 90, 44, 26, 26, 33],
       [ 9, 18, 72, 66, 12, 87, 19, 64, 19, 10],
       [72, 96, 22, 91, 85, 50, 51, 37, 16, 62],
       [71, 42, 68, 13, 50, 88, 55, 30, 21, 89],
       [ 3, 42, 88, 78, 75, 37, 85, 50, 35, 36],
       [54,  4, 44,  2, 81, 12, 17, 88, 63, 74],
       [63, 49, 72, 87, 68,  8, 26, 61, 21, 24],
       [38, 38, 72,  1, 48, 15, 42, 75, 49, 69],
       [61, 14, 54, 94, 41, 34, 52, 97, 40, 36]], dtype=uint8)

In [87]:
sorted([1,22,12,3])

[1, 3, 12, 22]

In [117]:
imread('/home/chengshun.wang/pjs/Semantic-Ray/data/scannet/scene0000_00/color/0.jpg').max()

255